# 성능올리기 위한 시도한 과정들

***1. Augmentation***

1.   좌우반전 이미지 생성
2.   -20~20도 5도씩 돌려서 이미지 생성


여러이미지를 생성


In [0]:
DATA_ROOT_TRAIN="/content/drive/My Drive/2019-ml-finalproject/train"
train_des=list()
train_labels=list()


for cls in tqdm(os.listdir(DATA_ROOT_TRAIN)): #진행도를 알려줌
  img_list = os.listdir(DATA_ROOT_TRAIN+'/'+cls)
  

  #백그라운드 적용 or 적용x
  
  if cls == 'BACKGROUND_Google' :
    label= 102
  else :
    label=(df_data.index[df_data[1]==cls]+1).tolist()[0]

  if cls == 'BACKGROUND_Google' :
    continue
  for img in img_list:

    image=Image.open(DATA_ROOT_TRAIN+'/'+cls+'/'+img)
    gray=image.convert("L") 
    for ang in range(-20, 20, 5):
      img2 = gray.rotate(ang)
      data = np.asarray(img2)
      train_des.append(data)
      train_labels.append(label)
      # 반전하기
    if cls != 'stop_sign':
      img2 = gray.transpose(Image.FLIP_LEFT_RIGHT)
      data = np.asarray(img2)
      train_des.append(data)
      train_labels.append(label)


**2. PCA**

  SIFT 적용 불가

In [0]:
from sklearn.decomposition import PCA
pca=PCA(200)

for i in range(len(train_des)):
  train_des[i]=np.ravel(train_des[i])
    
print(np.shape(train_des[4]))


gray=pca.fit_transform(train_des)
gray=pca.inverse_transform(gray)
print(np.shape(gray))

plt.imshow(gray[5].reshape(256,256),cmap='gray')

**3. Spatial Pyramid Matching**

In [0]:
from sklearn.svm import SVC

def histogramIntersection(M, N):
    m = M.shape[0]
    n = N.shape[0]
    result = np.zeros((m,n))
    for i in range(m):
        if i%100==1:
          print(i,"번째 실행중")
        for j in range(n):
            temp = np.sum(np.minimum(M[i], N[j]))
            result[i][j] = temp
    return result
gramMatrix = histogramIntersection(final_histo1, final_histo1)
print(gramMatrix.shape)
print(np.shape(final_label1))
final_clf = SVC(kernel='precomputed',C=1)
final_clf.fit(gramMatrix,final_label1)
predictMatrix = histogramIntersection(final_test_histo1, final_histo1)
result=final_clf.predict(predictMatrix)

In [0]:
predictMatrix = histogramIntersection(final_test_histo1, final_histo1)
result=final_clf.predict(predictMatrix)

In [0]:
import math

def extract_denseSIFT(img):
    disft_step_size = 8
    sift = cv2.xfeatures2d.SIFT_create()

    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, img.shape[0], disft_step_size)
                for x in range(0, img.shape[1], disft_step_size)]
    #strong feature
    descriptors = sift.compute(img, keypoints)[1]
    return descriptors


def getImageFeaturesSPM(L, img, kmeans, k):
    W = img.shape[1]
    H = img.shape[0]   
    h = []
    #h= np.zeros(k)

    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):                
                desc= extract_denseSIFT(img[y:y+h_step, x:x+w_step])

                predict = kmeans.predict(desc)
                histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
                #single level이 아닌 각 레벨마다 가중치부여
                #lv0: 1/4 lv1 : 1/4 lv2 : 1/2
                if l==0:
                  weight=2**(-2) 
                else:
                  weight = 2**(l-L-1) 

                #히스토그램 쭉 붙여서 SVM 돌리기
                h.append(weight*histo)

                #히스토그램 쌓아서 SVM 돌리기
                #h+=weight*histo
                x = x + w_step
            y = y + h_step
            
    hist = np.array(h).ravel()
    # 정규화 하기 or 안하기
    '''
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    '''

    return hist

def getHistogramSPM(L, data, kmeans, k):    
    x = []
    for i in (range(len(data))):
        if i%100==1:
          print(i,"번째 실행중")
        hist = getImageFeaturesSPM(L, data[i], kmeans, k)        
        x.append(hist)
    return np.array(x)

In [0]:
with open('/content/drive/My Drive/2019-ml-finalproject/test_des.pickle', 'rb') as f:
    test_des = pickle.load(f)
print(np.shape(test_des[0]))
final_test_histo1 = getHistogramSPM(2, test_des, kmeans1, k)
final_histo1=getHistogramSPM(2,train_des1,kmeans1,k)


In [0]:
clf = LinearSVC(random_state=0, C=0.0004304,class_weight='balanced')
clf.fit(final_histo1,final_label1)

result=clf.predict(final_test_histo1)